# EMIT: Classify interference type

This example shows how you can use PyAEDT to load an existing AEDT
project with an EMIT design and analyze the results to classify the
worst-case interference.


# Perform required imports

Perform required imports.


In [ ]:
import sys
from ansys.aedt.core.emit_core.emit_constants import InterfererType, ResultType, TxRxMode
from ansys.aedt.core import Emit
import ansys.aedt.core
import os
import ansys.aedt.core.generic.constants as consts
import subprocess

# Set AEDT version

Set AEDT version.


In [ ]:
aedt_version = "2024.2"

# Check to see which Python libraries have been installed
reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]


# Install required packages if they are not installed
def install(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])


# Install plotly library (if needed) to display legend and scenario matrix results (internet connection needed)
required_packages = ['plotly']
for package in required_packages:
    if package not in installed_packages:
        install(package)

# Import plotly library 
import plotly.graph_objects as go

# Define colors for tables
table_colors = {"green": '#7d73ca', "yellow": '#d359a2', "orange": '#ff6361', "red": '#ffa600', "white": '#ffffff'}
header_color = 'grey'

# Check for if emit version is compatible
if aedt_version <= "2023.1":
    print("Warning: this example requires AEDT 2023.2 or later.")
    sys.exit()

# Launch AEDT with EMIT

Launch AEDT with EMIT. The `Desktop` class initializes AEDT and starts
it on the specified version and in the specified graphical mode.


In [ ]:
non_graphical = False
new_thread = True
desktop = ansys.aedt.core.launch_desktop(aedt_version, non_graphical=non_graphical, new_desktop=new_thread)

path_to_desktop_project = ansys.aedt.core.downloads.download_file("emit", "interference.aedtz")
emitapp = Emit(non_graphical=False, new_desktop=False, project=path_to_desktop_project)

# Get all the radios in the project
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get lists of all transmitters and receivers in the project.
rev = emitapp.results.analyze()
tx_interferer = InterfererType().TRANSMITTERS
rx_radios = rev.get_receiver_names()
tx_radios = rev.get_interferer_names(tx_interferer)
domain = emitapp.results.interaction_domain()

if tx_radios is None or rx_radios is None:
    print("No receivers or transmitters are in the design.")
    sys.exit()

# Classify the interference

Iterate over all the transmitters and receivers and compute the power at
the input to each receiver due to each of the transmitters. Computes
which, if any, type of interference occurred.


In [ ]:
power_matrix = []
all_colors = []
all_colors, power_matrix = rev.interference_type_classification(domain, use_filter=False, filter_list=[])

# Save project and close AEDT

After the simulation completes, you can close AEDT or release it using
the `ansys.aedt.core.Desktop.force_close_desktop`{.interpreted-text
role="func"} method. All methods provide for saving the project before
closing.


In [ ]:
emitapp.save_project()
emitapp.release_desktop()

# Create a scenario matrix view

Create a scenario matrix view with the transmitters defined across the
top and receivers down the left-most column. The power at the input to
each receiver is shown in each cell of the matrix and color-coded based
on the interference type.


In [ ]:
def create_scenario_view(emis, colors, tx_radios, rx_radios):
    """Create a scenario matrix-like table with the higher received
    power for each Tx-Rx radio combination. The colors
    used for the scenario matrix view are based on the interference type."""

    all_colors = []
    for color in colors:
        col = []
        for cell in color:
            col.append(table_colors[cell])
        all_colors.append(col)

    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Tx/Rx</b>', '<b>{}</b>'.format(tx_radios[0]), '<b>{}</b>'.format(tx_radios[1])],
            line_color='darkslategray',
            fill_color='grey',
            align=['left', 'center'],
            font=dict(color='white', size=16)
        ),
        cells=dict(
            values=[
                rx_radios,
                emis[0],
                emis[1]],
            line_color='darkslategray',
            fill_color=['white', all_colors[0], all_colors[1]],
            align=['left', 'center'],
            height=25,
            font=dict(
                color=['darkslategray', 'black'],
                size=15)
        )
    )])
    fig.update_layout(
        title=dict(
            text='Interference Type Classification',
            font=dict(color='darkslategray', size=20),
            x=0.5
        ),
        width=600
    )
    fig.show()

# Generate a legend

Define the interference types and colors used to display the results of
the analysis.


In [ ]:
def create_legend_table():
    """Create a table showing the interference types."""
    classifications = ['In-band/In-band', 'Out-of-band/In-band',
                       'In-band/Out-of-band', 'Out-of-band/Out-of-band']
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Interference Type (Source/Victim)</b>'],
            line_color='darkslategray',
            fill_color=header_color,
            align=['center'],
            font=dict(color='white', size=16)
        ),
        cells=dict(
            values=[classifications],
            line_color='darkslategray',
            fill_color=[[table_colors['red'], table_colors['orange'], table_colors['yellow'], table_colors['green']]],
            align=['center'],
            height=25,
            font=dict(
                color=['darkslategray', 'black'],
                size=15)
        )
    )])
    fig.update_layout(
        title=dict(
            text='Interference Type Classification',
            font=dict(color='darkslategray', size=20),
            x=0.5
        ),
        width=600
    )
    fig.show()


if os.getenv("PYAEDT_DOC_GENERATION", "False") != "1":
    # Create a scenario view for all the interference types
    create_scenario_view(power_matrix, all_colors, tx_radios, rx_radios)

    # Create a legend for the interference types
    create_legend_table()